In [195]:
import pandas as pd

import warnings 
warnings.filterwarnings(action='ignore')
df=pd.read_csv('data/test.csv')
df.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴'],
      dtype='object')

## 기본컬럼 추가

In [196]:
len(df)

50

In [197]:
df.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지


In [198]:
df['식사가능인원'] = df['본사정원수'] - df['본사휴가자수'] - df['본사출장자수'] - df['현본사소속재택근무자수']
df.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,식사가능인원
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2355.0
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2319.0


## 날씨 데이터

In [199]:
weather = pd.read_csv('data/진주시_날짜_시간_날씨.csv')
print(len(weather))
weather.head(2)

45345


,지점,지점명,일시,기온(°C),강수량(mm),습도(%),적설(cm),일자,시간,연,월,일
0,192,진주,2016-02-01 00:00,0.5,NaN,44.0,NaN,2016-02-01,00:00,2016,2,1
1,192,진주,2016-02-01 01:00,-0.4,NaN,49.0,NaN,2016-02-01,01:00,2016,2,1


In [200]:
# 일단 결측치 전체 0으로 처리 후 작업
weather_f = weather.fillna(0)
weather_f.isna().sum().sum()

tmp = []
for x in weather_f['강수량(mm)']:
    if x == 0.0:
        tmp.append(0)
    else:
        tmp.append(1)
weather_f['강수여부'] = tmp
weather_f[weather_f['강수여부']==1]

lw = weather_f[['일자', '월', '일', '강수여부','기온(°C)','강수량(mm)']][weather_f['시간']== '12:00']
lw.columns = ['일자', '월', '일', '중식강수여부','중식기온','점심강수량']
lw.head(2)

# 저녁 날씨 데이터
dw = weather_f[['일자','월', '일','강수여부','기온(°C)','강수량(mm)']][weather_f['시간'] == '18:00']
dw.columns = ['일자', '월', '일', '석식강수여부','석식기온','저녁강수량']
dw.tail()

,일자,월,일,석식강수여부,석식기온,저녁강수량
45242,2021-04-05,4,5,0,16.1,0.0
45266,2021-04-06,4,6,0,20.0,0.0
45290,2021-04-07,4,7,0,17.0,0.0
45314,2021-04-08,4,8,0,17.3,0.0
45338,2021-04-09,4,9,0,13.8,0.0


In [201]:
df =pd.merge(df, lw, how='left')
df.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,식사가능인원,월,일,중식강수여부,중식기온,점심강수량
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2355.0,1,27,0,8.4,0.0
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2319.0,1,28,0,10.1,0.0


In [202]:
df =pd.merge(df, dw, how='left')
df.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,식사가능인원,월,일,중식강수여부,중식기온,점심강수량,석식강수여부,석식기온,저녁강수량
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2355.0,1,27,0,8.4,0.0,0,7.6,0.0
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2319.0,1,28,0,10.1,0.0,0,1.2,0.0


In [203]:
df.isna().sum().sum()

0

## 코로나

In [204]:
cor = pd.read_csv('data/코로나정보.csv')

cor_a=cor[cor['gubun']=='합계']

cor = cor_a.pivot_table(['incDec'], ['createDt'])
cor = cor.reset_index()
tmp = cor['createDt'].str.split(' ')

cor['일자'] = tmp.str.get(0) ## 0번째 년월일만 가지고 온다
cor = cor.drop(['createDt'], axis=1)

cor.columns = ['확진자수', '일자']

In [205]:
df =pd.merge(df, cor, how='left')
df.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,식사가능인원,월,일,중식강수여부,중식기온,점심강수량,석식강수여부,석식기온,저녁강수량,확진자수
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2355.0,1,27,0,8.4,0.0,0,7.6,0.0,559
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2319.0,1,28,0,10.1,0.0,0,1.2,0.0,497


In [206]:
len(df)

50

In [207]:
df = df[df.columns.drop(['조식메뉴'])]

In [208]:
import datetime
df['일자'] = pd.to_datetime(df['일자'])
df.head(2)
x = df['일자'][0]

In [209]:
x = df['일자'][0] ##시작일
be = []
af = []
i = 0
while 1:
    if x in df['일자'].values: #출근하는날
        be.append(0) #평번하게 출근하는날
        af.append(0)
        if len(af)>2 and be[-2] ==1:
            af.pop()
            af.append(1)
            
    else : #쉬는날 
        if be[-1] == 0: #쉬는날인데 전날이 평번한 출근이라면
            be[-1] =1
    
    x += datetime.timedelta(days=1) #1일씩 더한다
    if x == datetime.datetime(2021, 4, 10):
        break

len(af)

50

In [210]:
df['휴일전날'] = be
df['휴일다음날'] = af

In [211]:
df.loc[(df.요일=='월'),'휴일다음날'] = 0 #월요일 전체를 쉬는날로 한다
df.loc[(df.요일=='금'),'휴일전날'] = 0

In [212]:
df.columns = ['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식메뉴', '석식메뉴', '식사가능인원', '월', '일', '중식강수여부', '중식기온',
       '중식강수량', '석식강수여부', '석식기온', '석식강수량', '확진자수', '휴일전날', '휴일다음날']

## 메뉴카테고리

In [213]:
df_lunch = df[['일자']]
df_dinner = df[['일자']]

In [214]:
df.중식메뉴
lunch_menu = []

for day in range(len(df)):
    tmp = df.loc[day,'중식메뉴'].split(' ') # 스페이스로 구분
    tmp = ' '.join(tmp).split()    # 빈칸 제거

    for menu in tmp:
        if '(' in menu:
            tmp.remove(menu)
    lunch_menu.append(tmp) 

In [215]:
for bob in range(len(df)): # 밥은 큰 차이가 없을것같으므로 쌀밥은 다 밥으로 통일
    if '쌀밥' in lunch_menu[bob][0]:
        lunch_menu[bob][0] = '밥'

In [216]:
# 위에서 언급한 메인, 국, 반찬만 나누어주기

bob=[]
soup=[]
main=[]
menu=[]
for i in range(len(df)):
    if lunch_menu[i][0]:
        bob.append(lunch_menu[i][0])
    if lunch_menu[i][1]:
        soup.append(lunch_menu[i][1])
    if lunch_menu[i][2]:
        main.append(lunch_menu[i][2])
    if lunch_menu[i][3]:
        menu.append(lunch_menu[i][3])
    

In [217]:
df_lunch['밥'] = bob
df_lunch['국'] = soup
df_lunch['메인'] = main
df_lunch['반찬'] = menu

## 저녁

In [218]:
dinner_menu = []

for day in range(len(df)):
    tmp = df.loc[day,'석식메뉴'].split(' ') # 스페이스로 구분
    tmp = ' '.join(tmp).split()    # 빈칸 제거

    for menu in tmp:
        if '(' in menu:
            tmp.remove(menu)
    dinner_menu.append(tmp)

In [219]:
for bob in range(len(df)): # 밥은 큰 차이가 없을것같으므로 쌀밥은 다 밥으로 통일
    if '쌀밥' in dinner_menu[bob][0]:
        dinner_menu[bob][0] = '밥'

In [220]:
# 위에서 언급한 메인, 국, 반찬만 나누어주기

bob=[]
soup=[]
main=[]
menu=[]
for i in range(len(df)):
    if dinner_menu[i][0]:
        bob.append(dinner_menu[i][0])
    if dinner_menu[i][1]:
        soup.append(dinner_menu[i][1])
    if dinner_menu[i][2]:
        main.append(dinner_menu[i][2])
    if dinner_menu[i][3]:
        menu.append(dinner_menu[i][3])
    

In [221]:
df_dinner['밥'] = bob
df_dinner['국'] = soup
df_dinner['메인'] = main
df_dinner['반찬'] = menu

In [222]:
import numpy as np
df_lunch['국분류'] = np.nan
df_dinner['국분류'] = np.nan
df_lunch['메뉴분류'] = np.nan
df_dinner['메뉴분류'] = np.nan

In [223]:
# 국데이터에 있는 국류 분류
# 찌개, 국, 탕, 스프, 짬뽕, 우동, 라면, 국수, 냉면
from pandas import Series
for x in range(len(df_lunch['국'])):
    sm = Series(df_lunch['국'][x])
    if sm.str.contains('찌개')[0] == True:
            df_lunch['국분류'][x] = '찌개'
    elif sm.str.contains('국')[0] == True:
        df_lunch['국분류'][x] = '국'
    elif sm.str.contains('탕')[0] == True:
        df_lunch['국분류'][x] = '탕'
    elif sm.str.contains('스프')[0] == True:
        df_lunch['국분류'][x] = '스프'
    elif sm.str.contains('짬뽕')[0] == True:
        df_lunch['국분류'][x] = '국수'
    elif sm.str.contains('우동')[0] == True:
        df_lunch['국분류'][x] = '우동'
    elif sm.str.contains('개장')[0] == True:
        df_lunch['국분류'][x] = '국'        
    elif sm.str.contains('라면')[0] == True:
        df_lunch['국분류'][x] = '국수'
    elif sm.str.contains('면')[0] == True:
        df_lunch['국분류'][x] = '국수'
    elif sm.str.contains('수제비')[0] == True:
        df_lunch['국분류'][x] = '국'
    elif sm.str.contains('국수')[0] == True:
        df_lunch['국분류'][x] = '국수'
    elif sm.str.contains('냉면')[0] == True:
        df_lunch['국분류'][x] = '냉면'    
    elif sm.str.contains('죽')[0] == True:
        df_lunch['국분류'][x] = '죽'
    elif sm.str.contains('냉면')[0] == True:
        df_lunch['국분류'][x] = '냉면'
    elif sm.str.contains('미소시루')[0] == True:
        df_lunch['국분류'][x] = '국'
    elif sm.str.contains('소바')[0] == True:
        df_lunch['국분류'][x] = '국수'
    elif sm.str.contains('샤브')[0] == True:
        df_lunch['국분류'][x] = '탕'
    elif sm.str.contains('샤브')[0] == True:
        df_lunch['국분류'][x] = '탕'
    elif sm.str.contains('죽')[0] == True:
        df_lunch['국분류'][x] = '죽'
    elif sm.str.contains('찜')[0] == True:
        df_lunch['국분류'][x] = '찜'
    elif sm.str.contains('지리')[0] == True:
        df_lunch['국분류'][x] = '탕'


In [224]:
# 국데이터에 있는 국류 분류
# 찌개, 국, 탕, 스프, 짬뽕, 우동, 라면, 국수, 냉면
from pandas import Series
for x in range(len(df_dinner['국'])):
    sm = Series(df_dinner['국'][x])
    if sm.str.contains('찌개')[0] == True:
            df_dinner['국분류'][x] = '찌개'
    elif sm.str.contains('국')[0] == True:
        df_dinner['국분류'][x] = '국'
    elif sm.str.contains('탕')[0] == True:
        df_dinner['국분류'][x] = '탕'
    elif sm.str.contains('스프')[0] == True:
        df_dinner['국분류'][x] = '스프'
    elif sm.str.contains('짬뽕')[0] == True:
        df_dinner['국분류'][x] = '국수'
    elif sm.str.contains('우동')[0] == True:
        df_dinner['국분류'][x] = '우동'
    elif sm.str.contains('개장')[0] == True:
        df_dinner['국분류'][x] = '국'        
    elif sm.str.contains('라면')[0] == True:
        df_dinner['국분류'][x] = '국수'
    elif sm.str.contains('면')[0] == True:
        df_dinner['국분류'][x] = '국수'
    elif sm.str.contains('수제비')[0] == True:
        df_dinner['국분류'][x] = '국'
    elif sm.str.contains('국수')[0] == True:
        df_dinner['국분류'][x] = '국수'
    elif sm.str.contains('냉면')[0] == True:
        df_dinner['국분류'][x] = '냉면'    
    elif sm.str.contains('죽')[0] == True:
        df_dinner['국분류'][x] = '죽'
    elif sm.str.contains('냉면')[0] == True:
        df_dinner['국분류'][x] = '냉면'
    elif sm.str.contains('미소시루')[0] == True:
        df_dinner['국분류'][x] = '국'
    elif sm.str.contains('소바')[0] == True:
        df_dinner['국분류'][x] = '국수'
    elif sm.str.contains('샤브')[0] == True:
        df_dinner['국분류'][x] = '탕'
    elif sm.str.contains('샤브')[0] == True:
        df_dinner['국분류'][x] = '탕'
    elif sm.str.contains('죽')[0] == True:
        df_dinner['국분류'][x] = '죽'
    elif sm.str.contains('찜')[0] == True:
        df_dinner['국분류'][x] = '찜'
    elif sm.str.contains('지리')[0] == True:
        df_dinner['국분류'][x] = '탕'


In [225]:
df_lunch.loc[9,'국분류'] = '탕'

In [226]:
tmp = ['메인', '국', '밥', '반찬']
from pandas import Series
df_lunch['메뉴분류'] = df_lunch['메뉴분류'].fillna(0)
cnt = 0
for z in tmp:
    for x in range(len(df_lunch[z])):
        sm = Series(df_lunch[z][x])
        if df_lunch['메뉴분류'][x] == 0:
            if sm.str.contains('불고기')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('돈')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('쇠')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('갈비')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('장조림')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('탕수육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('목살')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('스테이크')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('소세지')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('햄')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('부대')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('제육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('궁중')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('미트')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('꼬지')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('순대')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('수육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('소고기')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('베이컨')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('함박')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('삼겹')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('너비아니')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('동파육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('만두')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('유린기')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('족발')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('오향장육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('비엔나')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('바베큐')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('두루치기')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('너비아니')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('등심')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('고기')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('등뼈')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('깐풍육')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('산적')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'
            elif sm.str.contains('육전')[0] == True:
                df_lunch['메뉴분류'][x] = '육류'


            elif sm.str.contains('닭')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('치킨')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('오리')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('계란')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('깐풍기')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('유린기')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('메추리알')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('달걀')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('삼계탕')[0] == True:
                df_lunch['메뉴분류'][x] = '조류'
            elif sm.str.contains('삼계탕')[0] == True:
                df_lunch['오므'][x] = '조류'
            
            
            
            
            
            
            
                
            #어류
            elif sm.str.contains('갈치')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('고등어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('가자미')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('꽁치')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('탕수어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('코다리')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('삼치')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('장어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('맛살')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('해물')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('동태')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('임연수')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('생선')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('굴비')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('조기')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('산슬')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('열기어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('적어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('황태')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('대구')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('참치')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('날치')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('골뱅이')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('꽁채')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            


            #연체류
            elif sm.str.contains('오징어')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('주꾸미')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('쭈꾸미')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('낙지')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            
            #갑각류
            elif sm.str.contains('게')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            elif sm.str.contains('새우')[0] == True:
                df_lunch['메뉴분류'][x] = '해산물'
            
            else:
                df_lunch['메뉴분류'][x] = '기타'
        

        

In [227]:
tmp = ['메인', '국', '밥', '반찬']
from pandas import Series
df_dinner['메뉴분류'] = df_dinner['메뉴분류'].fillna(0)
cnt = 0
for z in tmp:
    for x in range(len(df_dinner[z])):
        sm = Series(df_dinner[z][x])
        if df_dinner['메뉴분류'][x] == 0:
            if sm.str.contains('불고기')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('돈')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('쇠')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('갈비')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('장조림')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('탕수육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('목살')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('스테이크')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('소세지')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('햄')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('부대')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('제육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('궁중')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('미트')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('꼬지')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('순대')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('수육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('소고기')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('베이컨')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('함박')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('삼겹')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('너비아니')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('동파육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('만두')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('유린기')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('족발')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('오향장육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('비엔나')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('바베큐')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('두루치기')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('너비아니')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('등심')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('고기')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('등뼈')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('깐풍육')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('산적')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'
            elif sm.str.contains('육전')[0] == True:
                df_dinner['메뉴분류'][x] = '육류'


            elif sm.str.contains('닭')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('치킨')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('오리')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('계란')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('깐풍기')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('유린기')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('메추리알')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('달걀')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('삼계탕')[0] == True:
                df_dinner['메뉴분류'][x] = '조류'
            elif sm.str.contains('삼계탕')[0] == True:
                df_dinner['오므'][x] = '조류'
            
            
            
            
            
            
            
                
            #어류
            elif sm.str.contains('갈치')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('고등어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('가자미')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('꽁치')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('탕수어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('코다리')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('삼치')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('장어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('맛살')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('해물')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('동태')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('임연수')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('생선')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('굴비')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('조기')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('산슬')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('열기어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('적어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('황태')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('대구')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('참치')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('날치')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('골뱅이')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('꽁채')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            


            #연체류
            elif sm.str.contains('오징어')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('주꾸미')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('쭈꾸미')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('낙지')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            
            #갑각류
            elif sm.str.contains('게')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            elif sm.str.contains('새우')[0] == True:
                df_dinner['메뉴분류'][x] = '해산물'
            
            else:
                df_dinner['메뉴분류'][x] = '기타'
        

        

In [228]:
df_lunch = pd.merge(df,df_lunch[['일자','국분류', '메뉴분류']])

In [229]:
df_dinner = pd.merge(df,df_dinner[['일자','국분류','메뉴분류']])

In [230]:
df_dinner.loc[22,'국분류'] = '국'

In [231]:
df_lunch.drop(columns=['중식메뉴','석식메뉴', '일'],inplace=True)
df_dinner.drop(columns=['중식메뉴','석식메뉴', '일'],inplace=True)

In [232]:
df.to_csv('data/test_data.csv', index=False)
df_lunch.to_csv('data/test_lunch.csv', index=False)
df_dinner.to_csv('data/test_dinner.csv', index=False)